In [127]:
import requests
from bs4 import BeautifulSoup as BSoup
import os
import re
from itertools import compress
import time
from pathlib import Path

In [20]:
url = "https://projekter.aau.dk/projekter/da/studentthesis/search.html?search=" \
        "&advanced=true" \
        "&type=%2Fdk%2Fmasterthesis" \
        "&education=71383896" \
        "&ordering=studentProjectOrderByPublicationYear" \
        "&descending=true" \
        "&pageSize=500" \
        "&page=0"

In [16]:
url

'https://projekter.aau.dk/projekter/da/studentthesis/search.html?search=&advanced=true&type=%2Fdk%2Fmasterthesis&education=71383896&ordering=studentProjectOrderByPublicationYear&descending=true&pageSize=500&page=0'

In [22]:
response = requests.get(url, stream = True)

In [23]:
response.ok

True

In [26]:
source = response.text

In [28]:
sourceSoup = BSoup(source)

In [101]:
def get_urls(soup):
    #year = soup.get_text()
    
    sibling = soup.next_sibling
    sibling_class = sibling['class']
    
    urls = list()
    
    while 'portal_list_item' in sibling_class:
        urls.append(sibling.a.get('href'))
        sibling = sibling.next_sibling
        try:
            sibling_class = sibling['class']
        except:
            break
    
    return(urls)

['https://projekter.aau.dk/projekter/files/320352924/MMH_Speciale_MainDoc.pdf',
 'https://projekter.aau.dk/projekter/files/329935300/Hvordan_kan_vi_fa_dig_til_at_blive.pdf']

In [106]:
thesis_dict = dict()

for soup in sourceSoup.find_all(class_="portal_list_item_group"):
    year = soup.get_text()
    
    thesis_dict[year] = get_urls(soup)
    
print(thesis_dict.keys())

dict_keys(['2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2006'])


In [147]:
def dl_file(href, filepath = "./data/", reset_cache = False):
    try:
        filename = re.findall(r'(?<=\d{5}\/).*?\.\w{2,5}', href)[0]
    except:
        return
    
    if reset_cache == False:
        existing_filepath = Path(filepath + filename)
        if existing_filepath.is_file():
            print("\tFile {} already downloaded".format(filename), end = '\n')
            return
        else:
            reset_cache = True
    
    if reset_cache == True:
        response = requests.get(href)
    
        if response.ok:
            with open(filepath + filename, 'wb') as f:
                f.write(response.content)
                time.sleep(0.6)

            #print("file {} downloaded".format(filename))
        else:
            print("could not download file {}".format(filename))

datadir = "./data/"    
if not os.path.exists(datadir):
    os.makedirs(datadir)    

for year in list(thesis_dict.keys()):
    yeardir = datadir + year + "/"
    
    if not os.path.exists(yeardir):
        os.makedirs(yeardir)
    
    print("Downloading documents for year {}".format(year), end = '\n')
    c = 0
    for c, href in enumerate(thesis_dict[year]):
        print('Completed: {:.2f}%'.format(100.0*c/len(thesis_dict[year])), end = '\r')
        
        dl_file(href, filepath = yeardir)
        
        
    print("All documents downloaded", end = '\r')

	File MMH_Speciale_MainDoc.pdf already downloaded
	File Hvordan_kan_vi_fa_dig_til_at_blive.pdf already downloaded
	File Speciale_fardig.pdf already downloaded
	File SPECIALET_SAMLET__1_.pdf already downloaded
	File 010619SpecialeFinal.pdf already downloaded
	File Speciale___Magnus_Kamstrup_Mortensen___03.06 already downloaded
	File fardigt_speciale.pdf already downloaded
	File Speciale.pdf already downloaded
	File Et_opgor_med_negativ_social_kontrol.pdf already downloaded
	File Speciale___Faderskabsfortallinger_blandt_faglarte_fadre.pdf already downloaded
	File Julie_M.W._Jens already downloaded
	File SPECIALE.pdf already downloaded
	File SPECIALE____Fardigt.pdf already downloaded
	File Hvem_hjalper_mor_og_far___Omsorgsfortallinger_pdf.pdf already downloaded
	File Speciale___Maiken_Pedersen.pdf already downloaded
	File Speciale_EA.pdf already downloaded
	File Fardigt_speciale.pdf already downloaded
	File Okologisk_Speciale_maltid___Michael_.pdf already downloaded
	File Speciale___3.6.1